# On particle filters applied to electricity load forecasting : 3pm (15:00) model

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
import matplotlib as mtp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import truncnorm
import scipy.stats as stats
from scipy.stats import invgamma
from scipy import ndimage
import math
import time
import pickle
from datetime import date

In [2]:
%load_ext autoreload
%autoreload 2
from elec_forecast.bootstrap_filter_model import BootstrapFilterModel
from elec_forecast.rm_estimation import RmEstimation
from elec_forecast.particle_filter import ParticleFilter

### Load temperatures data 2015-2016

In [4]:
df_temp=pd.read_csv('data/temp_1516.csv',sep=',')
df_temp=df_temp.drop(df_temp.columns[0],axis=1)
df_temp=df_temp.sort_values(by='date')
df_temp.head()

,date,hour,t
0,2015-01-01,00:00,5.057414
1,2015-01-01,03:00,4.609138
2,2015-01-01,06:00,4.729828
3,2015-01-01,09:00,6.052241
4,2015-01-01,12:00,9.347069


In [5]:
#temperatures every day at 3pm
hour = "15:00"
temp_day = df_temp[df_temp.hour == hour]
#temp_day.head()
#Array vector of temperatures for each day of the year
T_h=list(temp_day.t)

### Load electricity data

In [6]:
mypath="data/"
df=pd.read_csv(mypath+"cdc_conso_daytypes.csv",sep=",",encoding='latin-1')
df=df.drop(["daytype"+str(i) for i in range(0,9)],axis=1)
df=df.drop(df.columns[0],axis=1)
df.index=range(0,len(df))
df=df.sort_values(by=['Date'])
df['Date - Heure']=pd.to_datetime(df['Date - Heure'])
df.sort_values(by=["Date - Heure"])
#truncate to the data from 2015
df=df[(df.Date >='2015-01-01')]
df.head()

,Date - Heure,Date,Heure,Consommation (MW),Qualite donnee,daytype
166957,2015-01-01 03:30:00,2015-01-01,04:30,64838,DÃ©finitive,6
153294,2015-01-01 18:30:00,2015-01-01,19:30,72562,DÃ©finitive,6
153293,2015-01-01 18:00:00,2015-01-01,19:00,71766,DÃ©finitive,6
152017,2015-01-01 01:00:00,2015-01-01,02:00,71134,DÃ©finitive,6
152018,2015-01-01 04:30:00,2015-01-01,05:30,64379,DÃ©finitive,6


### Set variables of electricity demand

In [7]:
df['Consommation']=df['Consommation (MW)']   #leave it in MW
#For prediction, every day at 3pm
consumption_day_ahead = list(df[df['Heure'] == hour]['Consommation'])

### Daytype

In [8]:
daytype=np.array(df[df['Heure']==hour]['daytype'])

## Implementation of Algorithm 3.10

## At time n=0:

### 1.Definition of MCMC initial model at n=0 - sample $X_0^{j} \sim \mu(x_0)$

### Option C: MCMC using MHA or Gibbs (TBA)

#### Freeze parameters from Zak's simulation in gibbs-parameters_init_v1

### Parameters

In [3]:
M=10**3 #number of particles
n_pred=715  #number of predictions

In [9]:
N_daytype=9
#k_day=npr.dirichlet(np.ones(N_daytype),1)
len_init=15
pred_forward=[0]

In [10]:
#Load pickle file
output_file ='data/parameters_init_20180113-135329.pkl'
pkl_file = open(output_file, 'rb')
parameters_init = pickle.load(pkl_file)
parameters_init

{'g_heat_init': array([-814623.94530988,   -1103.5042025 ,  -13509.35223552,
         -12839.18874712,  -13810.69394068,  -13621.95767042,
         -16305.45122647,  -16251.90232773,  -16229.3696953 ,
         -17229.49394215,  -13669.17597616,  -15726.25635544,
         -14808.97478176,  -14519.69267177,  -13655.86349596,
         -13546.63330023,  -13762.6351832 ,  -16223.92473751,
         -16231.78960435,  -17198.22294889,  -16084.04185055,
         -14859.33888566,  -15086.97652322,  -15659.32736924,
         -15025.21396291,  -16876.95500528,  -18016.91503558,
         -14386.93382501,  -14456.1228341 ,  -17258.13559834,
         -16399.27181928,  -14184.38234493,  -17037.65684651,
         -17577.66776037,  -15905.47860891,  -14955.89959227,
         -16811.49451007,  -15179.88058645,  -16273.87576028,
         -16666.00910714,  -16425.26301396,  -17107.27359641,
         -17374.00623292,  -13665.90412278,  -16588.1106145 ,
         -16457.00065552,  -16112.43401709,  -15547.969

In [11]:
s_init=parameters_init['s_init']
g_heat_init=parameters_init['g_heat_init']
sigma_s_init=parameters_init['sigma_s_init']
sigma_g_init=parameters_init['sigma_g_init']
x_init=parameters_init['x_init']
sigma2=parameters_init['sigma2']
u_h=parameters_init['u_h']
kappa=parameters_init['kappa']
w_init=parameters_init['w_init']

In [12]:
sigma2=np.ones(M)*3*10**7
sigma_g_init=np.ones(M)*10**4
sigma_s_init=np.ones(M)*10**4
kappa=np.ones(M)*1/9

In [13]:
w_init[0]=np.median(w_init)
x_init[0]=np.median(x_init)
g_heat_init[0]=np.median(g_heat_init)
s_init[0]=np.median(s_init)

### Part 2: regularize weights and x if necessary

In [14]:
pf = ParticleFilter(consumption_day_ahead, 
                    T_h, 
                    daytype, 
                    n_pred, 
                    M, 
                    x_init, 
                    w_init,
                    kappa,
                    u_h, 
                    sigma_s_init, 
                    sigma_g_init, 
                    s_init, 
                    g_heat_init, 
                    sigma2)

ESS of normalized weights= 862.234232
ESS>=0.5*M
new ESS= 862.234232


## Prediction and Filtering at time n>0

### 1. Sample $x^j_n \mid X^j_{n-1} $ for all j=1...M particles

In [15]:
start_time = time.time()
x_predict, ESS_calc = pf.predict(pred_forward, n_pred, len_init)
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'time' is not defined

In [ ]:
fig=plt.figure(figsize=(12,6))
plt.plot(range(1,n_pred),ESS_calc[0,1:n_pred]/M)
#plt.plot(range(1,n_pred),ESS_calc[1,1:n_pred]/M)
#plt.plot(range(1,n_pred),ESS_calc[2,1:n_pred]/M)

plt.title("Evolution of ESS",fontweight='bold')
plt.xlabel('forecasted day')
plt.xlim(0,n_pred)
plt.show()

In [ ]:
fig=plt.figure(figsize=(12,6))

plt.plot(range(n_pred),(x_predict[0,:]-consumption_day_ahead[:n_pred])/consumption_day_ahead[:n_pred],color='blue')
#plt.plot(range(n_pred),(x_predict[1,:]-consumption_day_ahead[1:n_pred+1])/consumption_day_ahead[1:n_pred+1],color='orange')
#plt.plot(range(n_pred),(x_predict[2,:]-consumption_day_ahead[2:n_pred+2])/consumption_day_ahead[2:n_pred+2],color='green')

plt.plot(range(n_pred-1),np.zeros(n_pred-1),color='red',linestyle='--')
plt.ylim(-1,1)
plt.xlim(0,n_pred)
plt.xlabel('forecasted day')
plt.title("Relative forecast error",fontweight='bold')
plt.show()

In [ ]:
fig=plt.figure(figsize=(12,6))
plt.plot(range(n_pred),x_predict[0,:])
#plt.plot(range(n_pred),x_predict[1,:])
#plt.plot(range(n_pred),x_predict[2,:])
plt.plot(range(n_pred),consumption_day_ahead[:n_pred],linestyle='--')
plt.xlabel('forecasted day')
plt.ylim(10**4,10**5)
plt.title("Electricity load forecast in W",fontweight='bold')
plt.show()

Goal: re-estimate the one-dimensional parameters ($\sigma_g,\sigma_s,\mu_h,\kappa,\sigma$)<br>
Proposal distribution for these parameters: truncated gaussian random walk<br>
We set kappa constant at 1/8



## PMMH

#### Initialization of parameters

In [ ]:
len_filter_pmmh = 730
len_iter_mha = 500

#### PMMH Algorithm

In [ ]:
start_time = time.time()
pf.pmmh(len_filter_pmmh, len_iter_mha, pred_forward, len_init)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
pf.sigma_g_list

In [ ]:
pf.sigma_s_list

In [ ]:
pf.sigma_list

In [ ]:
pf.u_h_list

In [ ]:
pf.accept_rate